# Bonus: Extended topics

In this notebook, we will work with the following topics:

1. Polars, an alternative to Pandas
1. Type hints

In [ ]:
import pandas as pd
import polars as pl

# Polars

[Polars](https://pola.rs) is a high performance dataframe package, written in the Rust programming language.
It generally has high performance, and it has some conveniences for working with large datasets that will not fit in memory, particularly when we only need a subset or aggregation of the data.

[Modern Polars](https://kevinheavey.github.io/modern-polars/) shows many examples of analogous Pandas and Polars code.

Below, let's rework our Pandas code from before.
Note that there are some efficiencies we could wring out here, but we're aiming for a similar flow to the original to make the comparison clearer.

In [ ]:
LOOKUP = {"Microsoft": "MSFT", "Google": "GOOG"}

firmyear = (
    pl.from_pandas(pd.read_stata("../data/firmyear.dta"))
    .with_columns(
        [
            pl.col("year").cast(pl.Int32),
            pl.col("count_of_employees").cast(pl.Int32),
            pl.col("name").replace(LOOKUP).alias("id_ticker"),
        ]
    )
    .rename({"count_of_employees": "size_emp"})
    .sort(["name", "year"])
    .with_columns(
        [
            pl.col("size_emp").diff().over("name").alias("size_emp_change"),
            pl.col("id_ticker").str.to_lowercase(),
        ]
    )
)

firmyear.head()

In [ ]:
stock = pl.read_csv("../data/stock.csv")

firmyear = firmyear.join(
    stock, how="left", left_on=["id_ticker", "year"], right_on=["tic", "yr"]
)

firmyear.head(6)

In [ ]:
msft_nyt = pl.read_csv("../data/msft_nyt.csv").with_columns(
    [pl.col("pub_date").str.strptime(pl.Date, "%Y-%m-%d %H:%M:%S")],
)

msft_nyt.head()

In [ ]:
def query_docs(data: pl.DataFrame, ticker: str, year: int) -> pl.DataFrame:
    filtered = data.filter(
        (pl.col("id_ticker") == ticker) & (pl.col("pub_date").dt.year() == year)
    )
    agg = filtered.select(
        [
            pl.col("word_count").mean().alias("wc_mean"),
            pl.col("word_count").sum().alias("wc_sum"),
        ]
    )
    return agg.with_columns(
        [pl.lit(ticker).alias("id_ticker"), pl.lit(year).alias("year")]
    )

In [ ]:
results = pl.concat(
    [
        query_docs(msft_nyt, row[0], row[1])
        for row in firmyear.select(["id_ticker", "year"]).rows()
    ]
)

firmyear = firmyear.join(results, on=["id_ticker", "year"], how="left")

firmyear.head(6)

As we can see above, we get substantially the same result, though it nicely omits the duplicated `tic` and `yr` columns from the pandas example.

# Type hints

Python supports something called [type hints](https://docs.python.org/3/library/typing.html), which are a way of annotating our code to express what types we think variables and arguments should be.

It is important to note that Python itself does not enforce these (cf. statically typed languages which do), and it will run code which violates the type hints with no warnings or errors.
However, many tools that run in VS Code or other editors can use type hints in conjunction with other tools to help us spot our own errors in logic and provide richer information for assisting us.

If you hover over the use of `query_docs()` above, you'll see annotated types for the arguments and the return type of the function itself.
If you go back to our original pandas code without type hints, you'll see those as `Unknown`.

In [ ]:
NO_HINT = "No type hint provided"
HINT: str = "Type hint!"

In [ ]:
def countdown_no_hint(count):
    if count < 1 or count > 5:
        count = 5
    for i in range(count, 0, -1):
        print(f"Counting down: {i}")
    print("Done!")

In [ ]:
countdown_no_hint(3)

In [ ]:
def countdown_hint(count: int) -> None:
    if count < 1 or count > 5:
        count = 5
    for i in range(count, 0, -1):
        print(f"Counting down: {i}")
    print("Done!")

In [ ]:
countdown_hint(3)

Similarly, if you type in this code,

```python
countdown_hint(3.0)
```

you will see a red underline noting that there is a type issue.
But, there's no such warning with this code:

```python
countdown_no_hint(3.0)
```


**My advice:** use type hints for functions whenever you can.
It's not always practical, because not every third-party package supports typing.
However, a lot of them do now, so it's often straightforward to do so.
This benefits you—via the better results from tools—and anyone reading your code who can more clearly see your intent.